# Instructor Do: Intro to VADER Sentiment

In [2]:
# Initial imports
from dotenv import load_dotenv
import os
from path import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/satishsurath/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# Load environment variables and read the News API key enviroment variable
load_dotenv()
api_key = os.getenv("NEWSAPI")

In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)



In [5]:
# Fetch all the news about Facebook Libra
libra_headlines = newsapi.get_everything(
    q="facebook AND libra",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Facebook Libra: {libra_headlines['totalResults']}")

# Show sample article
libra_headlines["articles"][0]



Total articles about Facebook Libra: 33


{'source': {'id': 'axios', 'name': 'Axios'},
 'author': 'Scott Rosenberg',
 'title': 'Facebook keeps pivoting to new projects — and then abandoning them',
 'description': 'Facebook\'s plan to wind down its podcast platform is the latest in a long line of projects the social network has launched with fanfare and then unceremoniously scuttled.Why it matters: "Launch, test, shut down" is a common pattern in tech\'s "fail fast" cultu…',
 'url': 'https://www.axios.com/2022/05/04/facebooks-endless-pivot-metaverse-podcasts',
 'urlToImage': 'https://images.axios.com/9wnl4CnD6xg67vSiqYA-yvPKwbs=/0x0:2545x1432/1366x768/2022/05/04/1651665611715.jpg',
 'publishedAt': '2022-05-04T10:00:32Z',
 'content': 'Facebook\'s plan to wind down its podcast platform is the latest in a long line of projects the social network has launched with fanfare and then unceremoniously scuttled.\r\nWhy it matters: "Launch, te… [+3317 chars]'}

In [7]:
# Create the Facebook Libra sentiment scores DataFrame
libra_sentiments = []

for article in libra_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        libra_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
libra_df = pd.DataFrame(libra_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
libra_df = libra_df[cols]

libra_df.head()



,date,text,compound,positive,negative,neutral
0,2022-05-04,Facebook's plan to wind down its podcast platf...,0.1531,0.075,0.000,0.925
1,2022-04-10,Not everyone gets along with everyone — while ...,0.0000,0.107,0.083,0.810
2,2022-05-04,Facebook's plan to wind down its podcast platf...,0.1531,0.075,0.000,0.925
3,2022-04-24,All of us have something negative in us and th...,-0.7037,0.036,0.149,0.815
4,2022-05-09,"Mark Zuckerberg, CEO of Facebooks parent compa...",0.4215,0.083,0.000,0.917


In [7]:
# Get descriptive stats from the DataFrame
libra_df.describe()



,compound,positive,negative,neutral
count,98.000000,98.000000,98.000000,98.000000
mean,0.120195,0.076031,0.047378,0.876612
std,0.429226,0.070121,0.065451,0.087655
min,-0.877900,0.000000,0.000000,0.629000
25%,-0.051600,0.000000,0.000000,0.812000
50%,0.076900,0.079500,0.000000,0.877500
75%,0.472225,0.108500,0.094000,0.939500
max,0.871800,0.308000,0.291000,1.000000
